In [19]:
import pandas as pd

In [20]:
data = pd.read_csv("../data/deputat_daily_presence.csv")

In [21]:
# First, add the date features

from fastai.tabular import add_datepart
data = add_datepart(data, 'date', drop=False).drop(columns=["Elapsed", 'Dayofyear'])

In [22]:
data.head()

,Unnamed: 0,deputat,party,present,date,Year,Month,Week,Day,Dayofweek,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start
0,1,АДЛЕН ШУКРИ ШЕВКЕД,ДПС,False,2019-12-06,2019,12,49,6,4,False,False,False,False,False,False
1,2,АЛБЕНА ВЛАДИМИРОВА НАЙДЕНОВА,ВОЛЯ,True,2019-12-06,2019,12,49,6,4,False,False,False,False,False,False
2,3,АЛЕКСАНДЪР ДИМИТРОВ ПАУНОВ,ПГБСП,True,2019-12-06,2019,12,49,6,4,False,False,False,False,False,False
3,4,АЛЕКСАНДЪР ИВАНОВ МАЦУРЕВ,ГЕРБ,False,2019-12-06,2019,12,49,6,4,False,False,False,False,False,False
4,5,АЛЕКСАНДЪР КОЙЧЕВ ИВАНОВ,ГЕРБ,False,2019-12-06,2019,12,49,6,4,False,False,False,False,False,False


In [23]:
def get_previous_presence(deputat_presence):
    deputat_presence = deputat_presence.sort_values('date')
    deputat_presence['present_last_ass'] = deputat_presence['present'].rolling(2).apply(lambda x: x.iloc[0])
    deputat_presence['present_last_5ass'] = deputat_presence['present'].rolling(6).apply(lambda x: x.iloc[:5].sum() / 5)
    return deputat_presence

data = data.groupby('deputat').apply(get_previous_presence)
    

In [26]:
data = data.reset_index(drop=True).drop(columns=['Unnamed: 0'])

In [27]:

data.to_csv("../data/deputat_daily_features.csv")

## Parliament level features

In [76]:
def party_level_presences(day_data):
    day_features = {}
    day_features['presence'] = day_data['present'].mean()
    for party in day_data.party.unique():
        day_features[f'{party}_presence'] = day_data.loc[day_data.party==party]['present'].mean()
    return pd.Series(day_features)


party_data = data.groupby('date').apply(party_level_presences).reset_index().sort_values('date')

In [77]:
party_data.sort_values('date')

,date,presence,ДПС_presence,ВОЛЯ_presence,ПГБСП_presence,ГЕРБ_presence,ОП_presence,НЕЗ_presence
0,2019-01-11,0.733333,0.68,0.500000,0.683544,0.778947,0.888889,0.500
1,2019-01-16,0.554167,0.60,0.500000,0.607595,0.452632,0.740741,0.500
2,2019-01-17,0.620833,0.64,0.666667,0.556962,0.652632,0.666667,0.500
3,2019-01-18,0.570833,0.48,0.250000,0.481013,0.705263,0.592593,0.500
4,2019-01-23,0.516667,0.00,0.333333,0.544304,0.600000,0.703704,0.500
...,...,...,...,...,...,...,...,...
146,2020-03-18,0.620833,0.88,0.083333,0.594937,0.557895,0.952381,0.750
147,2020-03-20,0.525000,0.80,0.000000,0.531646,0.431579,0.952381,0.375
148,2020-03-23,0.583333,0.56,0.000000,0.683544,0.589474,0.571429,0.500
149,2020-03-26,0.595833,0.48,0.416667,0.645570,0.652632,0.571429,0.125


In [78]:
party_data['next_day_presence'] = party_data['presence'].shift(-1)

In [81]:
party_data = add_datepart(party_data, 'date', drop=False).drop(columns=['Elapsed', 'Dayofyear', 'Day'])

In [82]:
party_data.to_csv("../data/parliament_daily_presence.csv")